In [ ]:
import os, sys
import pandas as pd
import torch

In [ ]:
!pip install deepface

     |████████████████████████████████| 62 kB 140 kB/s 
     |████████████████████████████████| 2.3 MB 9.6 MB/s 


In [ ]:
!pip install face_recognition

     |████████████████████████████████| 100.1 MB 26 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=0a1ee247e7fa4bf07383c456a46ffb74f5960e6d1f344c54e745e62411872bcd
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from drive.MyDrive.backend.all_models import predict_on_video, blink_on_video, detect_beard, detect_shades
from drive.MyDrive.backend.helper_functions import get_model

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2326528/45929032 bytes (5.1%)6004736/45929032 bytes (13.1%)9699328/45929032 bytes (21.1%)13352960/45929032 bytes (29.1%)16850944/45929032 bytes (36.7%)20504576/45929032 bytes (44.6%)24199168/45929032 bytes (52.7%)27697152/45929032 bytes (60.3%)31318016/45929032 bytes (68.2%)34766848/45929032 bytes (75.7%)38207488/45929032 bytes (83.2%)41836544/45929032 bytes (91.1%)

In [ ]:
# load df from csv
test_df = pd.read_csv("drive/MyDrive/backend/deepfake_test_updated.csv", header=0)
test_df

,Name,Real Labels,DFD (15 FPS),DFD (12 FPS),DFD (09 FPS),DFD (06 FPS),DFD (03 FPS),DFD (01 FPS),Blink (15 FPS) 0,Blink (15 FPS) 1,Blink (15 FPS) 2,Blink (15 FPS) 3,Beard: Age,Beard: Gender,Glasses
0,aassnaulhq.mp4,0,0.529217,0.536800,0.536746,0.550164,0.560346,0.548783,0.0,48.67,5.33,46.00,43.0,Man,9.12
1,aayfryxljh.mp4,0,0.005199,0.005302,0.004884,0.004623,0.003173,0.017791,0.0,0.00,88.67,11.33,47.0,Man,0.00
2,acazlolrpz.mp4,1,0.733870,0.722285,0.727390,0.749724,0.756411,0.740346,0.0,14.00,9.33,76.67,40.0,Man,0.00
3,adohdulfwb.mp4,0,0.088653,0.086411,0.090924,0.099174,0.125159,0.054809,0.0,0.00,92.67,7.33,33.0,Woman,0.00
4,ahjnxtiamx.mp4,1,0.903451,0.910799,0.885521,0.902039,0.924152,0.975536,0.0,0.00,58.67,41.33,42.0,Man,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,ztyuiqrhdk.mp4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,ztyvglkcsf.mp4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,zuwwbbusgl.mp4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,zxacihctqp.mp4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Change the directories as you see fit.

cwd = os.getcwd()
pre2 = cwd+"/drive/MyDrive/backend/data/test_videos"

# This is the directory of the 4 model outputs = 4 txt files.
# output_dir = "../src/pages/Home/VideoFilePlayer"

path_to_img="drive/MyDrive/backend/example_videos/temp/"
temp_img_shades=path_to_img+'p.png'
temp_img_beard=path_to_img+'o.png'

In [ ]:
# Use GPU, if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# Set up libraries
# sys.path.insert(0,cwd + "drive/MyDrive/backend/imports/blazeface")

# import libraries
from drive.MyDrive.backend.imports.blazeface.blazeface import BlazeFace

In [ ]:
# Load blazeface training weights from directory
facedet = BlazeFace().to(device)
facedet.load_weights(cwd + "/drive/MyDrive/backend/imports/blazeface/blazeface.pth")
facedet.load_anchors(cwd + "/drive/MyDrive/backend/imports/blazeface/anchors.npy")
_ = facedet.train(False)

In [ ]:
# Run all models

In [ ]:
for i in range(314,361):
  video_path = pre2+'/'+str(test_df['Name'].iloc[i])

  model_for_tests=get_model()
  blink_scores=blink_on_video(video_path, 15, facedet, model_for_tests)
  test_df.loc[i, 'Blink (15 FPS) 0'] = blink_scores[0]
  test_df.loc[i, 'Blink (15 FPS) 1'] = blink_scores[1]
  test_df.loc[i, 'Blink (15 FPS) 2'] = blink_scores[2]
  test_df.loc[i, 'Blink (15 FPS) 3'] = blink_scores[3]

  if(blink_scores[2]>0 or blink_scores[3]>0):
    beard_age, beard_gender=detect_beard(temp_img_beard)
    test_df.loc[i, 'Beard: Age'] = beard_age
    test_df.loc[i, 'Beard: Gender'] = beard_gender

    glasses_score=detect_shades(temp_img_shades)
    test_df.loc[i, 'Glasses'] = glasses_score

  else:
    test_df.loc[i, 'Beard: Age'] = -1
    test_df.loc[i, 'Beard: Gender'] = "N/A"
    test_df.loc[i, 'Glasses'] = -1

  test_df.to_csv("drive/MyDrive/backend/deepfake_test_updated.csv", index=False)

  print("\n---------"+str(i)+"---------\n")


58900480/58889256 [==============================] - 0s 0us/step

blink_on_video()
Amount of missing frames: 0.0%
Amount of unknown frames: 0.0%
Amount of open eyes frames: 72.66666666666667%
Amount of closed eyes frames: 27.333333333333332%
[0.0, 0.0, 72.67, 27.33]

detect_beard()
age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /root/.deepface/weights/age_model_weights.h5
100%|██████████| 539M/539M [02:43<00:00, 3.30MB/s]


gender_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: /root/.deepface/weights/gender_model_weights.h5
100%|██████████| 537M/537M [00:45<00:00, 11.8MB/s]
Action: gender: 100%|██████████| 2/2 [00:02<00:00,  1.16s/it]


   Age: 31
Gender: Man
There appears to be an adult male. Beards are hard to deepfake.

detect_shades()
49152/35363 [=========================================] - 0s 0us/step
Top 5 Object Detection Predictions
Band_Aid (9.94%)
seat_belt (4.52%)
cellular_telephone (3.02%)
wig (2.70%)
bow_tie (2.58%)
No sunglasses detected.

---------314---------


blink_on_video()
Amount of missing frames: 0.0%
Amount of unknown frames: 84.66666666666667%
Amount of open eyes frames: 6.0%
Amount of closed eyes frames: 9.333333333333334%
[0.0, 84.67, 6.0, 9.33]

detect_beard()


...

Action: gender: 100%|██████████| 2/2 [00:00<00:00,  3.15it/s]


   Age: 42
Gender: Man
There appears to be an adult male. Beards are hard to deepfake.

detect_shades()
Top 5 Object Detection Predictions
sunscreen (8.06%)
Band_Aid (6.71%)
bow_tie (5.08%)
shower_cap (4.91%)
sunglasses (4.26%)
There appears to be sunglasses. Larger glasses are hard to deepfake.

---------359---------


blink_on_video()
Amount of missing frames: 0.0%
Amount of unknown frames: 68.66666666666667%
Amount of open eyes frames: 4.666666666666667%
Amount of closed eyes frames: 26.666666666666668%
[0.0, 68.67, 4.67, 26.67]

detect_beard()


Action: gender: 100%|██████████| 2/2 [00:00<00:00,  3.17it/s]


   Age: 38
Gender: Man
There appears to be an adult male. Beards are hard to deepfake.

detect_shades()
Top 5 Object Detection Predictions
mask (20.80%)
beer_glass (5.56%)
lighter (4.41%)
red_wine (3.93%)
pop_bottle (3.17%)
No sunglasses detected.

---------360---------



<Figure size 432x288 with 0 Axes>

In [ ]:
for i in range(300,361):
  video_path = pre2+'/'+str(test_df['Name'].iloc[i])

  DFD_score_15=predict_on_video(video_path, 15, device, facedet)
  test_df.loc[i, 'DFD (15 FPS)'] = DFD_score_15

  DFD_score_12=predict_on_video(video_path, 12, device, facedet)
  test_df.loc[i, 'DFD (12 FPS)'] = DFD_score_12

  DFD_score_09=predict_on_video(video_path, 9, device, facedet)
  test_df.loc[i, 'DFD (09 FPS)'] = DFD_score_09

  DFD_score_06=predict_on_video(video_path, 6, device, facedet)
  test_df.loc[i, 'DFD (06 FPS)'] = DFD_score_06

  DFD_score_03=predict_on_video(video_path, 3, device, facedet)
  test_df.loc[i, 'DFD (03 FPS)'] = DFD_score_03

  DFD_score_01=predict_on_video(video_path, 1, device, facedet)
  test_df.loc[i, 'DFD (01 FPS)'] = DFD_score_01

  test_df.to_csv("drive/MyDrive/backend/deepfake_test_updated.csv", index=False)

  print("\n---------"+str(i)+"---------\n")



predict_on_video()
0.37973546981811523

predict_on_video()
0.3796703517436981

predict_on_video()
0.3894304037094116

predict_on_video()
0.37688156962394714

predict_on_video()
0.38786986470222473

predict_on_video()
0.39422956109046936

---------300---------


predict_on_video()
0.3963979482650757

predict_on_video()
0.4009389877319336

predict_on_video()
0.39979711174964905

predict_on_video()
0.40763887763023376

predict_on_video()
0.3917459547519684

predict_on_video()
0.3808097541332245

---------301---------


predict_on_video()
0.8796472549438477

predict_on_video()
0.8663275837898254

predict_on_video()
0.8761945962905884

predict_on_video()
0.8717116713523865

predict_on_video()
0.8892194628715515

predict_on_video()
0.8511640429496765

---------302---------


predict_on_video()
0.04352685436606407

predict_on_video()
0.0525163970887661

predict_on_video()
0.04596559703350067

predict_on_video()
0.05104949697852135

predict_on_video()
0.04452354460954666

predict_on_video()
0

In [ ]:
# eyeblink -> NOT a deepfake if 55<open<98
# beard -> NOT a deepfake if 20yr+ male
# shades -> NOT a deepfake if no shades detected
# eyeblink+beard
# eyeblink+shades
# beard+shades
# eyeblink+beard+shades